## Download urbanized areas' street networks

Using census tigerline shapefile of 2017 urban areas

In [1]:
import config
import geopandas as gpd
import json
import logging as lg
import networkx as nx
import os
import osmnx as ox
import pandas as pd
import time

ox.config(use_cache=True,
          log_file=True,
          log_console=True,
          log_filename='download-urbanized_areas',
          cache_folder=config.urbanized_areas_cache_folder)

print(ox.__version__)
print(nx.__version__)

0.7.4
2.1


In [2]:
network_type = 'drive'
retain_all = True
simplify = True

input_folder = 'input_data/urbanized_areas' #tiger place shapefiles
output_folder_shapefile = config.urbanized_areas_shapefile_folder #where to save graph shapefiles
output_folder_graphml = config.urbanized_areas_graphml_folder #where to save graphml files

## Load shapes

In [3]:
with open('input_data/states_by_fips.json') as f:
    fips_to_state = json.load(f)

st_fips = {v['abbreviation']:k for k, v in fips_to_state.items()}

In [4]:
gdf = gpd.read_file(input_folder).sort_values(by='ALAND10', ascending=True)
len(gdf)

3601

In [5]:
# remove the tiny "urban clusters" to retain only "urbanized areas"
urbanized_areas = gdf[~gdf['NAMELSAD10'].str.contains('Urban Cluster')]
len(urbanized_areas)

497

In [6]:
# tigerline data is epsg:4269, but osm uses epsg:4326, so project it
print(urbanized_areas.crs)
urbanized_areas = urbanized_areas.to_crs({'init':'epsg:4326'})
urbanized_areas.crs

{'init': 'epsg:4269'}


{'init': 'epsg:4326'}

In [7]:
urbanized_areas[['ALAND10', 'NAMELSAD10', 'GEOID10']].tail()

,ALAND10,NAMELSAD10,GEOID10
1593,4852213624,"Boston, MA--NH--RI Urbanized Area",09271
2546,5131722319,"Philadelphia, PA--NJ--DE--MD Urbanized Area",69076
3250,6323669758,"Chicago, IL--IN Urbanized Area",16264
3002,6850603579,"Atlanta, GA Urbanized Area",03817
2321,8934395320,"New York--Newark, NY--NJ--CT Urbanized Area",63217


## Prepare queries

In [8]:
def make_fips_state(s):
    start = s.find(', ') + 2
    state = s[start:start+2]
    fips_state = '{}_{}'.format(st_fips[state], state)
    return fips_state
    
urbanized_areas['fips_state'] = urbanized_areas['NAMELSAD10'].map(make_fips_state)

In [9]:
def make_geoid_name(row):
    return '{}_{}'.format(row['GEOID10'], row['NAMELSAD10']).replace(',', '').replace(' ', '_').replace('/', '_')

urbanized_areas['geoid_name'] = urbanized_areas.apply(make_geoid_name, axis=1)

In [10]:
queries = urbanized_areas.apply(lambda row: {'fips_state':row['fips_state'],
                                             'geoid_name':row['geoid_name'],
                                             'polygon':row['geometry']}, axis=1).tolist()
queries[0]

{'fips_state': '06_CA',
 'geoid_name': '22987_Delano_CA_Urbanized_Area',
 'polygon': <shapely.geometry.multipolygon.MultiPolygon at 0x217980b4940>}

## Download and save graphs

In [11]:
start_time = time.time()

for query in queries:
    try:
        graphml_folder = '{}/{}'.format(output_folder_graphml, query['fips_state'])
        graphml_file = '{}.graphml'.format(query['geoid_name'])
        shapefile_folder = '{}/{}'.format(output_folder_shapefile, query['fips_state'])
        shapefile_file = query['geoid_name']
        
        if not os.path.exists('{}/{}'.format(graphml_folder, graphml_file)):
            
            G = ox.graph_from_polygon(polygon=query['polygon'],
                                      network_type=network_type,
                                      name=query['geoid_name'],
                                      retain_all=retain_all,
                                      simplify=simplify)
            
            ox.save_graph_shapefile(G, folder=shapefile_folder, filename=shapefile_file)
            ox.save_graphml(G, folder=graphml_folder, filename=graphml_file)
            
    except Exception as e:
        ox.log('"{}" failed: {}'.format(query['geoid_name'], e), level=lg.ERROR)
        
print('Finished making graphs in {:,.1f} seconds'.format(time.time() - start_time))

Finished making graphs in 38,503.0 seconds


In [12]:
#39,000 seconds last time